In [1]:
from scipy import stats

import scipy
scipy.__version__


'1.12.0'

In [2]:
# Get data and helper functions from tables.ipynb
%run tables.ipynb

test_accuracy$s168740-cl1-cf128-dm-bn0.csv
test_accuracy$s168740-cl1-cf128-dm-bn1.csv
test_accuracy$s168740-cl1-cf128-dn-bn0.csv
test_accuracy$s168740-cl1-cf128-dn-bn1.csv
test_accuracy$s168740-cl1-cf128-ds-bn0.csv
test_accuracy$s168740-cl1-cf128-ds-bn1.csv
test_accuracy$s168740-cl1-cf16-dm-bn0.csv
test_accuracy$s168740-cl1-cf16-dm-bn1.csv
test_accuracy$s168740-cl1-cf16-dn-bn0.csv
test_accuracy$s168740-cl1-cf16-dn-bn1.csv
test_accuracy$s168740-cl1-cf16-ds-bn0.csv
test_accuracy$s168740-cl1-cf16-ds-bn1.csv
test_accuracy$s168740-cl1-cf32-dm-bn0.csv
test_accuracy$s168740-cl1-cf32-dm-bn1.csv
test_accuracy$s168740-cl1-cf32-dn-bn0.csv
test_accuracy$s168740-cl1-cf32-dn-bn1.csv
test_accuracy$s168740-cl1-cf32-ds-bn0.csv
test_accuracy$s168740-cl1-cf32-ds-bn1.csv
test_accuracy$s168740-cl1-cf64-dm-bn0.csv
test_accuracy$s168740-cl1-cf64-dm-bn1.csv
test_accuracy$s168740-cl1-cf64-dn-bn0.csv
test_accuracy$s168740-cl1-cf64-dn-bn1.csv
test_accuracy$s168740-cl1-cf64-ds-bn0.csv
test_accuracy$s168740-cl1-cf

In [3]:
models, results = get_averages(exp_data, 'test_accuracy', filter='', printStats=False)
test_accuracy_results = {}
for i in range(len(models)):
    test_accuracy_results[models[i]] = results[i]

In [4]:
# E1 significance testing
for group in ['cf16-dn-bn0','cf32-dn-bn0','cf64-dn-bn0','cf128-dn-bn0']:
    group_results = []
    for k, v in test_accuracy_results.items():
        if not k.endswith(group): continue
        group_results.append(v)
    print(f'# ------ {group} ------ #')
    print(f'Inputs: {group_results}')
    print(f'Step 1: {stats.f_oneway(*group_results)}')
    # print(f'group means: {np.mean(group_results, axis=1)}')
    print(f'Step 2: t-test for cl5-{group} > cl1-{group}: {stats.ttest_ind(group_results[4], group_results[0], alternative="greater")}')
    

# ------ cf16-dn-bn0 ------ #
Inputs: [[0.787804901599884, 0.795121967792511, 0.7585365772247314, 0.790243923664093, 0.792682945728302], [0.8073170781135559, 0.792682945728302, 0.7829268574714661, 0.8024390339851379, 0.790243923664093], [0.8097561001777649, 0.8487805128097534, 0.8414633870124817, 0.8073170781135559, 0.8390243649482727], [0.8439024686813354, 0.8487805128097534, 0.8463414907455444, 0.8390243649482727, 0.8512195348739624], [0.8585366010665894, 0.8487805128097534, 0.8731707334518433, 0.8512195348739624, 0.8560975790023804]]
Step 1: F_onewayResult(statistic=30.970411692186133, pvalue=2.5880217947964322e-08)
Step 2: t-test for cl5-cf16-dn-bn0 > cl1-cf16-dn-bn0: TtestResult(statistic=9.152994853052036, pvalue=8.184284337897325e-06, df=8.0)
# ------ cf32-dn-bn0 ------ #
Inputs: [[0.8024390339851379, 0.8048780560493469, 0.800000011920929, 0.8073170781135559, 0.79756098985672], [0.8268292546272278, 0.8121951222419739, 0.8097561001777649, 0.8195121884346008, 0.8219512104988098], 

In [5]:
models, results = get_generalization_gap(exp_data, filter='', printStats=False)
test_gg_results = {}
for i in range(len(models)):
    test_gg_results[models[i]] = results[i]

In [6]:
# E2 significance testing
ttest_results_e2 = []
for cl in range(5):
    for cf in ['cf16','cf32','cf64','cf128']:
        model_bn = f'cl{cl+1}-{cf}-dn-bn1'
        model_dropout = f'cl{cl+1}-{cf}-dm-bn0'
        
        bn_results = test_gg_results[model_bn]
        dropout_results = test_gg_results[model_dropout]
        ttest = stats.ttest_ind(bn_results, dropout_results)
        ttest_results_e2.append(ttest)
        print(f'# ------ {model_bn} and {model_dropout} ------ #')
        print(f'Inputs: {[bn_results,dropout_results]}')
        print(ttest)
        
significance_level = 0.05

overall_cond = all(x.pvalue < significance_level for x in ttest_results_e2)
print(f'\n>>> All p-values below level of signifance (a={significance_level}): {overall_cond}')
        

# ------ cl1-cf16-dn-bn1 and cl1-cf16-dm-bn0 ------ #
Inputs: [[0.011713743209838867, 0.027713418006896973, 0.032885611057281494, 0.037366896867752075, 0.01872551441192627], [-0.023354709148406982, -0.02881869673728943, -0.017968833446502686, -0.006314754486083984, -0.020064353942871094]]
TtestResult(statistic=7.528568986457854, pvalue=6.74312499754556e-05, df=8.0)
# ------ cl1-cf32-dn-bn1 and cl1-cf32-dm-bn0 ------ #
Inputs: [[0.018937915563583374, 0.03234592080116272, 0.025905489921569824, 0.017457187175750732, 0.022110074758529663], [-0.022515177726745605, -0.03429266810417175, -0.010990887880325317, -0.0212591290473938, -0.028997600078582764]]
TtestResult(statistic=9.875350333130248, pvalue=9.32076136839103e-06, df=8.0)
# ------ cl1-cf64-dn-bn1 and cl1-cf64-dm-bn0 ------ #
Inputs: [[0.03795459866523743, 0.016178131103515625, 0.03424948453903198, 0.027027159929275513, 0.04500865936279297], [-0.01911330223083496, -0.03678974509239197, -0.015743494033813477, -0.007835179567337036, -0.